# emu

> Fill in a module description here

In [ ]:
#| default_exp emu

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from sepia.SepiaModel import SepiaModel
from sepia.SepiaData import SepiaData
from sepia.SepiaPredict import SepiaEmulatorPrediction
# from sepia.SepiaPredict import SepiaFullPrediction
# from sepia.SepiaPredict import SepiaXvalEmulatorPrediction
# from sepia.SepiaSharedThetaModels import SepiaSharedThetaModels
import numpy as np

In [ ]:
#| export

def emulate(sepia_model:SepiaModel=None, # Input data in SEPIA format
        input_params:np.array=None, #Input parameter array 
       ) -> tuple: # 2 np.array of mean and (0.05,0.95) quantile in prediction
    
    if len(input_params.shape) == 1:
        ip = np.expand_dims(input_params, axis=0)
        
    else:
        ip = input_params
        
    pred_samples= sepia_model.get_samples(numsamples=10)
        
    pred = SepiaEmulatorPrediction(t_pred=ip, samples=pred_samples, model=sepia_model)
    
    pred_samps = pred.get_y()
    
    pred_mean = np.mean(pred_samps, axis=0).T
    pred_err = np.quantile(pred_samps, [0.05, 0.95], axis=0).T
    
    return pred_mean, pred_err

In [ ]:
#| export

def model_load_all(): 
    '''
    model_list = []
    
    for z_index in range(51):
        sepia_model = gp_load(sepia_model, model_filename)
        model_list.append(sepia_model)
    return model_list
    '''
    pass
 

In [ ]:
#| export

def emu_redshift(input_params_and_redshift):
    
    '''
        if (z == 0):
            # No redshift interpolation for z=0
            GPm, PCAm = model_load(snap_ID=LAST_SNAP, nRankMax=DEFAULT_PCA_RANK)
            Pk_interp = gp_emu(GPm, PCAm, [Omh2, ns, s8, fR0, n])
        
        
        else:
            # Linear interpolation between z1 < z < z2
            snap_idx_nearest = (np.abs(z_all - z)).argmin()
            if (z > z_all[snap_idx_nearest]):
                snap_ID_z1 = snap_idx_nearest - 1
            else:
                snap_ID_z1 = snap_idx_nearest
            snap_ID_z2 = snap_ID_z1 + 1

            GPm1, PCAm1 = model_load(snap_ID=snap_ID_z1, nRankMax=DEFAULT_PCA_RANK)
            Pk_z1 = gp_emu(GPm1, PCAm1, [Omh2, ns, s8, fR0, n])
            z1 = z_all[snap_ID_z1]

            GPm2, PCAm2 = model_load(snap_ID=snap_ID_z2, nRankMax=DEFAULT_PCA_RANK)
            Pk_z2 = gp_emu(GPm2, PCAm2, [Omh2, ns, s8, fR0, n])
            z2 = z_all[snap_ID_z2]

            Pk_interp = np.zeros_like(Pk_z1)
            Pk_interp = Pk_z2 + (Pk_z1 - Pk_z2)*(z - z2)/(z1 - z2)
    return Pk_interp, kvals
    
    '''
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()